In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision.transforms.functional import to_pil_image
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import os
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from torch.utils.data import Dataset
from PIL import Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

# transforms 정의하기
h, w = 64, 64
mean = (0.5, 0.5, 0.5)
std = (0.5, 0.5, 0.5)

transform = transforms.Compose([
                    transforms.Resize((h,w)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean, std)
])

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.images = os.listdir(root)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, self.images[idx])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image


In [ ]:
batch_size = 64
dataset = ImageDataset(root='/content/drive/MyDrive/aipro/dataset', transform= transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batch_size, shuffle=True)

In [ ]:
# 파라미터 정의
params = {'nz':100, # noise 수
          'ngf':64, # generator에서 사용하는 conv filter 수
          'ndf':64, # discriminator에서 사용하는 conv filter 수
          'img_channel':3, # 이미지 채널
          }

In [ ]:
# Generator: noise를 입력받아 가짜 이미지를 생성합니다.
class Generator(nn.Module):
    def __init__(self, params):
        super().__init__()
        nz = params['nz'] # noise 수, 100
        ngf = params['ngf'] # conv filter 수
        img_channel = params['img_channel'] # 이미지 채널

        self.dconv1 = nn.ConvTranspose2d(nz,ngf*8,4, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(ngf*8)
        self.dconv2 = nn.ConvTranspose2d(ngf*8,ngf*4, 4, stride=2, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(ngf*4)
        self.dconv3 = nn.ConvTranspose2d(ngf*4,ngf*2,4,stride=2,padding=1,bias=False)
        self.bn3 = nn.BatchNorm2d(ngf*2)
        self.dconv4 = nn.ConvTranspose2d(ngf*2,ngf,4,stride=2,padding=1,bias=False)
        self.bn4 = nn.BatchNorm2d(ngf)
        self.dconv5 = nn.ConvTranspose2d(ngf,img_channel,4,stride=2,padding=1,bias=False)

    def forward(self,x):
        x = F.relu(self.bn1(self.dconv1(x)))
        x = F.relu(self.bn2(self.dconv2(x)))
        x = F.relu(self.bn3(self.dconv3(x)))
        x = F.relu(self.bn4(self.dconv4(x)))
        x = torch.tanh(self.dconv5(x))
        return x

# check
x = torch.randn(1,100,1,1, device=device)
model_gen = Generator(params).to(device)
out_gen = model_gen(x)
print(out_gen.shape)


In [ ]:
# Discriminator: 진짜 이미지와 가짜 이미지를 식별합니다.
class Discriminator(nn.Module):
    def __init__(self,params):
        super().__init__()
        img_channel = params['img_channel'] # 3
        ndf = params['ndf'] # 64

        self.conv1 = nn.Conv2d(img_channel,ndf,4,stride=2,padding=1,bias=False)
        self.conv2 = nn.Conv2d(ndf,ndf*2,4,stride=2,padding=1,bias=False)
        self.bn2 = nn.BatchNorm2d(ndf*2)
        self.conv3 = nn.Conv2d(ndf*2,ndf*4,4,stride=2,padding=1,bias=False)
        self.bn3 = nn.BatchNorm2d(ndf*4)
        self.conv4 = nn.Conv2d(ndf*4,ndf*8,4,stride=2,padding=1,bias=False)
        self.bn4 = nn.BatchNorm2d(ndf*8)
        self.conv5 = nn.Conv2d(ndf*8,1,4,stride=1,padding=0,bias=False)

    def forward(self,x):
        x = F.leaky_relu(self.conv1(x),0.2)
        x = F.leaky_relu(self.bn2(self.conv2(x)),0.2)
        x = F.leaky_relu(self.bn3(self.conv3(x)),0.2)
        x = F.leaky_relu(self.bn4(self.conv4(x)),0.2)
        x = torch.sigmoid(self.conv5(x))
        return x.view(-1,1)

# check
x = torch.randn(16,3,64,64,device=device)
model_dis = Discriminator(params).to(device)
out_dis = model_dis(x)
print(out_dis.shape)


In [ ]:
# 가중치 초기화
def initialize_weights(model):
    classname = model.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(model.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(model.weight.data, 1.0, 0.02)
        nn.init.constant_(model.bias.data, 0)

# 가중치 초기화 적용
model_gen.apply(initialize_weights);
model_dis.apply(initialize_weights);

In [ ]:
# 손실 함수 정의
loss_func = nn.BCELoss()

# 최적화 함수
from torch import optim
lr = 2e-4
beta1 = 0.5
beta2 = 0.999

opt_dis = optim.Adam(model_dis.parameters(),lr=lr,betas=(beta1,beta2))
opt_gen = optim.Adam(model_gen.parameters(),lr=lr,betas=(beta1,beta2))


In [ ]:

model_gen.train()
model_dis.train()

batch_count=0
num_epochs=200
start_time = time.time()
nz = params['nz'] # 노이즈 수 100
loss_hist = {'dis':[],
             'gen':[]}

for epoch in range(num_epochs):
    for xb in dataloader:
        ba_si = xb.shape[0]

        xb = xb.to(device)
        yb_real = torch.Tensor(ba_si,1).fill_(1.0).to(device)
        yb_fake = torch.Tensor(ba_si,1).fill_(0.0).to(device)

        # generator
        model_gen.zero_grad()
        z = torch.randn(ba_si,nz,1,1).to(device) # noise
        out_gen = model_gen(z) # 가짜 이미지 생성
        out_dis = model_dis(out_gen) # 가짜 이미지 식별

        g_loss = loss_func(out_dis,yb_real)
        g_loss.backward()
        opt_gen.step()

        # discriminator
        model_dis.zero_grad()
        out_dis = model_dis(xb) # 진짜 이미지 식별
        loss_real = loss_func(out_dis,yb_real)

        out_dis = model_dis(out_gen.detach()) # 가짜 이미지 식별
        loss_fake = loss_func(out_dis,yb_fake)

        d_loss = (loss_real + loss_fake) / 2
        d_loss.backward()
        opt_dis.step()

        loss_hist['gen'].append(g_loss.item())
        loss_hist['dis'].append(d_loss.item())

        batch_count += 1
        if batch_count % 64 == 0:
            print('Epoch: %.0f, G_Loss: %.6f, D_Loss: %.6f, time: %.2f min' %(epoch, g_loss.item(), d_loss.item(), (time.time()-start_time)/60))

In [ ]:
# loss history
plt.figure(figsize=(10,5))
plt.title('Loss Progress')
plt.plot(loss_hist['gen'], label='Gen. Loss')
plt.plot(loss_hist['dis'], label='Dis. Loss')
plt.xlabel('batch count')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# 가중치 저장
path2models = '/content/drive/MyDrive/aipro'
path2weights_gen = os.path.join(path2models, 'weights_gen.pt')
path2weights_dis = os.path.join(path2models, 'weights_dis.pt')

#torch.save(model_gen.state_dict(), path2weights_gen)
#torch.save(model_dis.state_dict(), path2weights_dis)

In [ ]:
# 가중치 불러오기
weights = torch.load(path2weights_gen)
model_gen.load_state_dict(weights)

# evalutaion mode
model_gen.eval()

# fake image 생성
with torch.no_grad():
    fixed_noise = torch.randn(1, 100,1,1, device=device)
    label = torch.randint(0,10,(16,), device=device)
    img_fake = model_gen(fixed_noise).detach().cpu()
print(img_fake.shape)

In [ ]:
with torch.no_grad():
    fixed_noise  = torch.randn(16,100,1,1, device = device)
    img_fake = model_gen(fixed_noise).detach().cpu()
print(img_fake.shape)


이 코드에서 img_fake에 대해 0.5를 곱하고 0.5를 더하는 것은 이미지의 픽셀 값 범위를 조절하는 목적을 가지고 있습니다.

GAN의 생성된 이미지는 보통 -1에서 1 사이의 값으로 정규화되어 있습니다. 이는 GAN이 -1부터 1까지의 범위에서 픽셀 값을 생성하도록 훈련되었기 때문입니다. 하지만 대부분의 이미지 시각화 도구 및 라이브러리는 0에서 1의 범위의 값으로 픽셀 값을 기대합니다.

따라서 0.5*img_fake + 0.5는 생성된 이미지의 픽셀 값을 -1에서 1의 범위에서 0에서 1의 범위로 변환하여 시각화를 수행하기 위한 것입니다. 이 작업을 통해 이미지를 정확한 범위로 변환하여 시각화할 수 있게 됩니다

In [ ]:
plt.imshow(to_pil_image(0.5*img_fake[0] + 0.5), cmap = 'gray')
plt.axis('off')

In [ ]:
# 가짜 이미지 시각화
plt.figure(figsize=(10,10))
for ii in range(16):
    plt.subplot(4,4,ii+1)
    plt.imshow(to_pil_image(0.5*img_fake[ii]+0.5), cmap='gray')
    plt.axis('off')

In [ ]:
add_epochs = 100

for epoch in range(add_epochs):
    for xb in dataloader:
        ba_si = xb.shape[0]

        xb = xb.to(device)
        yb_real = torch.Tensor(ba_si,1).fill_(1.0).to(device)
        yb_fake = torch.Tensor(ba_si,1).fill_(0.0).to(device)

        # generator
        model_gen.zero_grad()
        z = torch.randn(ba_si,nz,1,1).to(device) # noise
        out_gen = model_gen(z) # 가짜 이미지 생성
        out_dis = model_dis(out_gen) # 가짜 이미지 식별

        g_loss = loss_func(out_dis,yb_real)
        g_loss.backward()
        opt_gen.step()

        # discriminator
        model_dis.zero_grad()
        out_dis = model_dis(xb) # 진짜 이미지 식별
        loss_real = loss_func(out_dis,yb_real)

        out_dis = model_dis(out_gen.detach()) # 가짜 이미지 식별
        loss_fake = loss_func(out_dis,yb_fake)

        d_loss = (loss_real + loss_fake) / 2
        d_loss.backward()
        opt_dis.step()

        loss_hist['gen'].append(g_loss.item())
        loss_hist['dis'].append(d_loss.item())

        batch_count += 1
        if batch_count % 64 == 0:
            print('Epoch: %.0f, G_Loss: %.6f, D_Loss: %.6f, time: %.2f min' %(epoch, g_loss.item(), d_loss.item(), (time.time()-start_time)/60))

DCGAN은 입력 image의 크기를 64 x 64로 crop해서 사용하잖아. 만약 입력 input의 크기를 늘린다면, 생성 이미지의 화질 증가에 도움이 될까?

ChatGPT
일반적으로 DCGAN과 같은 GAN 모델은 고정된 입력 이미지 크기를 사용합니다. 입력 이미지의 크기를 늘리는 것이 항상 화질을 향상시킬 것이라고 단언하기는 어렵습니다. 큰 입력 이미지 크기는 몇 가지 고려해야 할 측면이 있습니다:

계산 비용: 큰 입력 이미지 크기는 더 많은 계산 비용을 필요로 합니다. 모델의 파라미터와 메모리 사용량이 늘어날 뿐만 아니라, 학습 및 생성에 더 많은 리소스가 필요합니다.

데이터의 다양성: 큰 입력 이미지는 더 다양한 특징을 포함할 수 있습니다. 그러나 이것이 항상 더 나은 결과를 가져다주지는 않습니다. 작은 크기의 이미지에서도 충분히 복잡한 패턴을 학습할 수 있기 때문입니다.

모델 복잡성: 더 큰 입력 이미지는 일반적으로 더 복잡한 모델을 필요로 합니다. 따라서 모델의 복잡성을 증가시키면서도 안정적인 학습이 보장되어야 합니다.

적절한 해상도 선택: 생성하려는 이미지의 용도에 따라 적절한 입력 이미지 해상도를 선택해야 합니다. 예를 들어, 작은 이미지에서 세부적인 구조를 보고 싶다면 큰 입력 이미지가 필요할 수 있습니다.

화질을 높이기 위해서는 위의 문제들을 고려하면서 적절한 trade-off를 찾아야 합니다. 실험을 통해 최적의 설정을 찾고, 필요한 경우에는 모델이나 데이터를 조정하여 화질을 향상시킬 수 있습니다.